# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.metrics import accuracy_score
import cvxpy as cp


def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

# Ridge regression

In [22]:
def fit_svd(X, y, alpha=1.0):
    """SVD approach"""
    N, d = X.shape
    
    U, S, Vt = np.linalg.svd(X)
    Sm = np.vstack([np.diag(S), np.zeros((N - d, d))])
    R = U @ Sm
    
    return Vt.T @ np.linalg.inv(R.T @ R + alpha * np.eye(d)) @ R.T @ y

In [23]:
w = fit_svd(X_train, y_train, alpha=0.1)

In [24]:
sk_model = Ridge(alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  1.7215104810280023e-13


# Newton Method - Logistic regression (IRLS)

In [25]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

data = load_breast_cancer()
X_train2, y_train2 = data.data, data.target
w2 = np.random.randn(X_train2.shape[1]) * 0.0001
b2 = np.random.randn(1) * 0.0001

In [27]:
class LogisticRegressor():
    def __init__(self):
        self.w = None

    def train(self, X, y, num_iters=100):
        N, d = X.shape
        
        self.w = np.zeros(d)
        s = np.zeros(N)
        z = np.zeros(N)
        
        for _ in range(num_iters):
            for n in range(N):
                an = self.w @ X[n]
                un = sigmoid(an)
                s[n] = un * (1 - un)
                z[n] = y[n] - un
            S = np.diag(s)
            self.w += np.linalg.inv(X.T @ S @ X) @ X.T @ z
    
    def predict(self, X):
        return X @ self.w > 0

In [35]:
class LogisticRegressor():
    def __init__(self):
        self.w = None

    def train(self, X, y, num_iters=100):
        N, d = X.shape
        
        self.w = np.zeros(d)
        
        for _ in range(num_iters):
            a = X @ self.w
            u = sigmoid(X @ self.w)
            s = u * (1 - u)
            z = y - u
            S = np.diag(s)
            self.w += np.linalg.inv(X.T @ S @ X) @ X.T @ z
            
    
    def predict(self, X):
        return X @ self.w > 0

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_acc = accuracy_score(sk_pred, y_train2)

model = LogisticRegressor()
model.train(X_train2, y_train2, num_iters=10)
pred = model.predict(X_train2)
model_acc = accuracy_score(pred, y_train2)

print("Accuracy scikit-learn:", sk_acc)
print("Accuracy IRLS model :", model_acc)
assert sk_acc - model_acc < 0.01

Accuracy scikit-learn: 0.9876977152899824
Accuracy IRLS model : 0.9947275922671354


# Constrained optimization CVXPY

Résoudre avec CVXPY

$$\min_{\theta \in \mathbb{R}^3} \frac{1}{2} (\theta_1^2 + \theta_2^2 + \theta_3^2)$$
$$\text{s.c. }$$
$$\theta_1 + \theta_2 + 2\theta_3 - 1 = 0$$
$$\theta_1 + 4\theta_2 + 2\theta_3 - 3 = 0$$


In [32]:
X = cp.Variable(3)

objective = cp.Minimize(cp.sum_squares(X) / 2)
constraints = [
    X @ np.array([1, 1, 2]) - 1 == 0,
    X @ np.array([1, 1, 2]) - 1 == 0
]

problem = cp.Problem(objective, constraints)
problem.solve()

print("Solution: ", X.value)

Solution:  [0.16666667 0.16666667 0.33333333]
